In [1]:
import altair as alt

from traffic.core import Traffic
from traffic.core.projection import EuroPP
import pandas as pd
from tqdm import tqdm

import torch

import warnings
warnings.filterwarnings("ignore")

In [2]:
t = Traffic.from_file("../deep_traffic_generation/data/traffic_15.pkl")
t

,count
flight_id,
ABP721_057,200
SWR134J_9747,200
SWR134J_9730,200
SWR134J_9731,200
SWR134J_9732,200
SWR134J_9733,200
SWR134J_9734,200
SWR134J_9736,200
SWR134J_9737,200


### Douglas-Peucker

In [3]:
simplified_t = (
    t
    .simplify(tolerance=1e3)
    .eval(desc="")
)
simplified_t

  0%|          | 0/3968 [00:00<?, ?it/s]

,count
flight_id,
SWR96C_17751,14
SWR122W_9296,14
DLH6NT_2732,13
SWR116W_8827,13
SWR77E_16703,13
SWR161_10900,13
SWR169A_11191,13
SWR109W_8524,12
SWR124P_9381,12


In [4]:
simplified_t.to_csv("./outputs/dp_ref.csv")

### Navigational Points

In [5]:
navpoints_t = pd.concat([
    flight
    .compute_navpoints()
    .assign(
        flight_id=flight.flight_id,
        callsign=flight.callsign,
        icao24=flight.icao24,
        registration=flight.registration,
        typecode=flight.typecode,
        latitude_0=flight.data.iloc[0].latitude,
        longitude_0=flight.data.iloc[0].longitude,
        altitude_0=flight.data.iloc[0].altitude,
        track_0=flight.data.iloc[0].track,
        groundspeed_0=flight.data.iloc[0].groundspeed,
    )
    .drop(columns=[
        "altitude",
        "description",
        "frequency",
        "magnetic_variation"
    ])
    for flight in tqdm(t)
])
navpoints_t.head()

100%|██████████| 3968/3968 [50:28<00:00,  1.31it/s]


,start,stop,duration,navaid,distance,shift_mean,shift_meanp,name,type,latitude,...,flight_id,callsign,icao24,registration,typecode,latitude_0,longitude_0,altitude_0,track_0,groundspeed_0
0,2019-10-31 08:45:10+00:00,2019-10-31 08:46:40+00:00,0 days 00:01:30,RILAX,4.725168,0.076809,0.096809,RILAX,FIX,47.942864,...,ABP721_057,ABP721,49d044,OK-SUN,E35L,48.118069,8.727722,13775.0,219.436058,357.358055
1,2019-10-31 08:47:08+00:00,2019-10-31 08:50:04+00:00,0 days 00:02:56,38LOC,19.598984,0.122979,0.142979,38LOC,FIX,47.394067,...,ABP721_057,ABP721,49d044,OK-SUN,E35L,48.118069,8.727722,13775.0,219.436058,357.358055
2,2019-10-31 08:54:48+00:00,2019-10-31 08:55:31+00:00,0 days 00:00:43,ZH064,9.207336,0.114647,0.134647,ZH064,FIX,47.522556,...,ABP721_057,ABP721,49d044,OK-SUN,E35L,48.118069,8.727722,13775.0,219.436058,357.358055
3,2019-10-31 08:56:09+00:00,2019-10-31 09:00:29+00:00,0 days 00:04:20,D337B,0.548766,0.039502,0.059502,D337B,FIX,47.488086,...,ABP721_057,ABP721,49d044,OK-SUN,E35L,48.118069,8.727722,13775.0,219.436058,357.358055
0,2019-10-04 05:19:38+00:00,2019-10-04 05:21:15+00:00,0 days 00:01:37,SONGI,8.188006,0.068770,0.088770,SONGI,FIX,47.777778,...,ACA802_062,ACA802,c058bf,C-GHPU,B788,48.128677,8.544960,12675.0,184.398705,365.075019


In [7]:
navpoints_t.to_csv("./outputs/nav_ref.csv")